In [ ]:
import cv2
from cvzone.HandTrackingModule import HandDetector
import numpy as np
import math
import time

# Initialize video capture and hand detector
cap = cv2.VideoCapture(0, cv2.CAP_AVFOUNDATION)
detector = HandDetector(maxHands=1)

# Constants for image processing
offset = 20
imgSize = 300
folder = "Data/K"
counter = 0

while True:
    # Read a frame from the video capture
    success, img = cap.read()
    
    # Detect hands in the frame
    hands, img = detector.findHands(img)

    if hands:
        hand = hands[0]
        x, y, w, h = hand["bbox"]

        # Create a white background for better visualization
        imgWhite = np.ones((imgSize, imgSize, 3), np.uint8) * 255

        # Crop and resize the hand region for better presentation
        if 0 < w < img.shape[1] and 0 < h < img.shape[0] and 0 < x < img.shape[1] and 0 < y < img.shape[0]:
            imgCrop = img[y - offset:y + h + offset, x - offset:x + w + offset]

            if not imgCrop.size == 0:
                aspectRatio = h / w 
                if aspectRatio > 1:
                    k = imgSize / h
                    wCal = math.ceil(k * w)
                    imgResize = cv2.resize(imgCrop, (wCal, imgSize))
                    wGap = math.ceil((imgSize - wCal) / 2)
                    imgWhite[:, wGap:wGap + wCal] = imgResize
                else:
                    k = imgSize / w
                    hCal = math.ceil(k * h)
                    imgResize = cv2.resize(imgCrop, (imgSize, hCal))
                    hGap = math.ceil((imgSize - hCal) / 2)
                    imgWhite[hGap:hGap + hCal, :] = imgResize

                # Display intermediate images for debugging
                cv2.imshow("ImageCrop", imgCrop)
                cv2.imshow("ImageWhite", imgWhite)

    # Display the original image
    cv2.imshow("Image", img)

    # Check for key presses
    key = cv2.waitKey(1) & 0xFF
    if key == ord('m'):
        break
    elif key == ord('s'):
        # Save processed image and increment counter
        counter += 1
        cv2.imwrite(f'{folder}/Image_{time.time()}.jpg', imgWhite)
        print(counter)

# Release the video capture and close all windows
cap.release()
cv2.destroyAllWindows()